In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
from torch.optim import Adam
import gym
import time

In [9]:
from spinup import models
core = models

IndentationError: expected an indented block (ppo.py, line 339)

In [ ]:

#from spinup.utils.logx import EpochLogger
from spinup.utils.mpi_pytorch import setup_pytorch_for_mpi, sync_params, mpi_avg_grads
from spinup.utils.mpi_tools import mpi_fork, mpi_avg, proc_id, mpi_statistics_scalar, num_procs

from spinup.algos.ppo.ppo import ppo

In [4]:
class Config(object):
    def __init__(self):
        return None
args = Config()
args.env="CartPole-v1"
args.algorithm="ppo"
args.name=f"{args.env}_{args.algorithm}"
args.gpu=0
args.seed=0
args.hid=64
args.l=2
args.gamma=0.99
args.cpu=4
args.steps=4000
args.epochs=50

In [6]:
#mpi_fork(args.cpu)  # run parallel code with mpi

ppo(lambda : gym.make(args.env), actor_critic=core.MLPActorCritic,
    ac_kwargs=dict(hidden_sizes=[args.hid]*args.l), gamma=args.gamma, 
    seed=args.seed, steps_per_epoch=args.steps, epochs=args.epochs)

NameError: name 'ppo' is not defined